Downloading necessary libraries

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md
!pip install imbalanced-learn
# !pip install --upgrade scikit-learn

# the scikit version to be loaded are different for normal classifier and bertweet classifier
# for normal classifier run without upgrading

Importing Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
from imblearn.over_sampling import RandomOverSampler
import spacy
import numpy as np
import numpy as np
import pandas as pd
import os
import glob

Loading Dataset

In [ ]:

data=pd.read_csv("/kaggle/input/bst-300k/behaviour_simulation_train.csv")
# test_data_company=pd.read_csv("/kaggle/input/test-set-adobe/behaviour_simulation_test_company.csv",encoding='latin-1')
# test_data_time=pd.read_csv("/kaggle/input/test-set-adobe/behaviour_simulation_test_time.csv" ,encoding='latin-1')


In [10]:
test_data_company=pd.read_excel("/kaggle/input/testset/behaviour_simulation_test_company.xlsx")
test_data_time=pd.read_excel("/kaggle/input/testset/behaviour_simulation_test_time.xlsx")

In [14]:
test_data_company=test_data_company.drop("media",axis=1)

In [15]:
test_data_time=test_data_time.drop("media",axis=1)

In [19]:
test_data_time

,id,date,content,username,inferred company
0,1,2021-11-08 00:40:49,"Andres, a Pharmacy Manager at Store 4669 in Pi...",WalmartWorld,walmart
1,2,2021-05-09 07:51:21,Happy #EuropeDay! We are always stronger by wo...,SimonHarrisTD,td
2,3,2022-11-22 05:30:05,"Election Results, BVAS Report Established Over...",IndependentNGR,independent
3,4,2021-09-22 21:52:08,American Olympic swimmer Elizabeth Beisel will...,CBCOlympics,cbc
4,5,2022-04-15 13:00:27,No need to hunt ‘em or hope to find ‘em in a b...,TimHortonsUS,tim hortons
...,...,...,...,...,...
9995,9996,2021-04-27 11:28:28,𝐊𝐚𝐢𝐳𝐞𝐧 - The essence of continuous improvement...,TGR_WEC,toyota
9996,9997,2021-10-10 10:08:29,JACK'S BACK! 🤩\n\nBest of luck to our Reserve ...,WilliamsRacing,williams
9997,9998,2022-10-15 05:13:00,[Special Saturday Hearing in #SupremeCourt]\n\...,barandbench,bar
9998,9999,2022-05-26 00:42:30,Obama has crossed the line. Don’t make this re...,w_terrence,williams


Loading Classifier

In [7]:
# Load the spaCy model with word vectors
nlp = spacy.load("en_core_web_md")

In [5]:
clf=joblib.load("/kaggle/input/classifiers/random_forest_classifier_bst-300k(299k).joblib")

Loading Embeddings

In [8]:
def predict_likes_class(index, dataframe, text_model, usr_model, com_model, date_model, classifier):
    # Extract the relevant information for the given index
    content = dataframe.loc[index, 'content']
    username = dataframe.loc[index, 'username']
    company = dataframe.loc[index, 'inferred company']
    date = dataframe.loc[index, 'date']

    # Get word embeddings for the input features
    content_embedding = text_model(content).vector
    usr_embedding = usr_model(username).vector
    com_embedding = com_model(company).vector
    date_embedding = date_model(date).vector

    # Combine the embeddings with other features
    input_features = np.concatenate([content_embedding, usr_embedding, com_embedding, date_embedding], axis=None)

    # Make the prediction using the trained classifier
    predicted_class = classifier.predict([input_features])[0]

    return predicted_class

Dont Run these cells if you want to predict with Normal Classifier

In [43]:
# Set the path to the directory containing your .npy files
input_directory = "/kaggle/input/adobe-test-embeddings"

# List all .npy files in the directory
npy_files = glob.glob(os.path.join(input_directory, '*.npy'))


In [47]:
npy_files.sort()
npy_files

['/kaggle/input/adobe-test-embeddings/content-test-company-date.npy',
 '/kaggle/input/adobe-test-embeddings/content-test-company-inferred-company.npy',
 '/kaggle/input/adobe-test-embeddings/content-test-company-username.npy',
 '/kaggle/input/adobe-test-embeddings/content-test-company.npy',
 '/kaggle/input/adobe-test-embeddings/content-test-time-date.npy',
 '/kaggle/input/adobe-test-embeddings/content-test-time-inferred-company.npy',
 '/kaggle/input/adobe-test-embeddings/content-test-time-username.npy',
 '/kaggle/input/adobe-test-embeddings/content-test-time.npy',
 '/kaggle/input/adobe-test-embeddings/full-content-test-company.npy',
 '/kaggle/input/adobe-test-embeddings/full-content-test-time.npy']

In [ ]:
# for company data
# Load each .npy file and concatenate them
arrays = [np.load(file) for file in npy_files[3:4]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_text = pd.DataFrame(combined_array, columns=column_names)


arrays = [np.load(file) for file in npy_files[0:1]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_date = pd.DataFrame(combined_array, columns=column_names)



arrays = [np.load(file) for file in npy_files[1:2]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_com = pd.DataFrame(combined_array, columns=column_names)



arrays = [np.load(file) for file in npy_files[2:3]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_usr = pd.DataFrame(combined_array, columns=column_names)


In [ ]:
# for time data
# Load each .npy file and concatenate them
arrays = [np.load(file) for file in npy_files[7:8]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_text = pd.DataFrame(combined_array, columns=column_names)


arrays = [np.load(file) for file in npy_files[4:5]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_date = pd.DataFrame(combined_array, columns=column_names)




arrays = [np.load(file) for file in npy_files[5:6]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_com = pd.DataFrame(combined_array, columns=column_names)





arrays = [np.load(file) for file in npy_files[6:7]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_usr = pd.DataFrame(combined_array, columns=column_names)


Loading Bertweet Classifier (Not imp)

In [ ]:
clf_bt=joblib.load("/kaggle/input/berttweetclassifier/random_forest_model_bertTweet(290k)_1.joblib")

In [ ]:
def predict_likes_class_berttweet(model, index,text,date,com,usr):
    # Create a DataFrame with the input features
    input_data = pd.DataFrame({
        'content': [],
        'inferred company': [],
        'date': [],
        'username': []
    })

    # Get word embeddings for the 'Content' column
    content_embedding = text[index:index+1]
    usr_embedding = usr[index:index+1]
    com_embedding = com[index:index+1]
    date_embedding = date[index:index+1]

    # Combine embeddings with other features
    input_combined = pd.concat([input_data.drop('content', axis=1), content_embedding], axis=1)
    input_combined = pd.concat([input_combined.drop('date', axis=1), date_embedding], axis=1)
    input_combined = pd.concat([input_combined.drop('inferred company', axis=1), com_embedding], axis=1)
    input_combined = pd.concat([input_combined.drop('username', axis=1), usr_embedding], axis=1)

    # Make predictions
    prediction = model.predict(input_combined)

    return prediction

end=100
for index in range(1000,1000+end):
    predicted_class = predict_likes_class(clf_bt, index,df_text,df_date,df_com,df_usr)
    print(f'Predicted Likes Class: {predicted_class[0]}')
    print(f"actual likes were {df2.iloc[index]['likes']}")


In [48]:
data

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
...,...,...,...,...,...,...,...
299995,299996,2019-09-07 16:18:10,0,Barcelona Star Expected To Return Against Vale...,IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
299996,299997,2018-02-23 11:24:36,46,Kjeld Nuis of #NED is golden again... This tim...,CBCOlympics,[Video(thumbnailUrl='https://pbs.twimg.com/med...,cbc
299997,299998,2020-11-11 20:18:15,261,Grateful 🙌🏾 to have <mention> officially join ...,TeamCoachBuzz,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
299998,299999,2019-10-29 10:44:00,119,The US Air Force's mysterious space plane just...,Independent,[Video(thumbnailUrl='https://pbs.twimg.com/med...,independent


Predicting Classes (Normal Classifier)

In [55]:
# upload the csv here to predict the classes here
# youll have to run this function once for unseen time and once for unseen company and edit accordingly
index_to_predict=0
final=10000
index=[]
classPredicted=[]

# for unseen test company change from test_data_time to test_data_company

for i in range(index_to_predict,index_to_predict+final): 
    if(i%100==0):
        print(i)
    predicted_class_label = predict_likes_class(i, test_data_time, nlp, nlp, nlp, nlp, clf)
    index.append(i)
    classPredicted.append(predicted_class_label)
    
ak2=pd.DataFrame({"index":index,"classByNormalClassifier_time":classPredicted})
ak2

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


,index,classByNormalClassifier_time
0,0,0-100
1,1,100-1000
2,2,0-100
3,3,100-1000
4,4,0-100
...,...,...
9995,9995,0-100
9996,9996,100-1000
9997,9997,0-100
9998,9998,100-1000


In [56]:
ak2.to_csv("classifiedByNormalClassifier_time_testdata.csv",index=False)

In [57]:
s=pd.read_csv("/kaggle/working/classifiedByNormalClassifier_time_testdata.csv")

In [58]:
s

,index,classByNormalClassifier_time
0,0,0-100
1,1,100-1000
2,2,0-100
3,3,100-1000
4,4,0-100
...,...,...
9995,9995,0-100
9996,9996,100-1000
9997,9997,0-100
9998,9998,100-1000
